In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.cluster import KMeans



reg_season_detailed_pd = pd.read_csv('data/RegularSeasonDetailedResults_Prelim2018.csv')
reg_season_detailed_pd.columns = ['Season', 'Daynum', 'Wteam', 'Wscore', 'Lteam', 'Lscore', 'Wloc',
       'Numot', 'Wfgm', 'Wfga', 'Wfgm3', 'Wfga3', 'Wftm', 'Wfta', 'Wor', 'Wdr',
       'Wast', 'Wto', 'Wstl', 'Wblk', 'Wpf', 'Lfgm', 'Lfga', 'Lfgm3', 'Lfga3',
       'Lftm', 'Lfta', 'Lor', 'Ldr', 'Last', 'Lto', 'Lstl', 'Lblk', 'Lpf']
reg_season_detailed_pd.columns
seasons_pd = pd.read_csv('data/Seasons.csv')
teams_pd = pd.read_csv('data/Teams.csv')
teamList = teams_pd['Team_Name'].tolist()
tourney_compact_pd = pd.read_csv('data/TourneyCompactResults.csv')
tourney_detailed_pd = pd.read_csv('data/TourneyDetailedResults.csv')
tourney_seeds_pd = pd.read_csv('data/TourneySeeds.csv')
tourney_seeds_pd.columns = ['Season', 'Seed', 'Team_Id']
tourney_slots_pd = pd.read_csv('data/TourneySlots.csv')
ncaa_tourney_compact_pd = pd.read_csv('data/NCAATourneyCompactResults.csv')
teams_pd_2 = pd.read_csv('data/Teams-2.csv')

In [3]:
reg_season_detailed_pd['Wfg%'] = reg_season_detailed_pd['Wfgm'] / reg_season_detailed_pd['Wfga']
reg_season_detailed_pd['Lfg%'] = reg_season_detailed_pd['Lfgm'] / reg_season_detailed_pd['Lfga']
reg_season_detailed_pd['Wfg%3'] = reg_season_detailed_pd['Wfgm3'] / reg_season_detailed_pd['Wfga3']
reg_season_detailed_pd['Lfg%3'] = reg_season_detailed_pd['Lfgm3'] / reg_season_detailed_pd['Lfga3']
reg_season_detailed_pd['Wft%'] = reg_season_detailed_pd['Wftm'] / reg_season_detailed_pd['Wfta']
reg_season_detailed_pd['Lft%'] = reg_season_detailed_pd['Lftm'] / reg_season_detailed_pd['Lfta']

#WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF

In [4]:
reg_season_detailed_2018_pd = reg_season_detailed_pd[reg_season_detailed_pd['Season'] == 2018]

In [5]:
reg_season_detailed_2018_pd.head(3)

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lto,Lstl,Lblk,Lpf,Wfg%,Lfg%,Wfg%3,Lfg%3,Wft%,Lft%
76636,2018,11,1104,82,1272,70,N,0,26,57,...,17,7,3,22,0.456140,0.383333,0.407407,0.117647,0.678571,0.611111
76637,2018,11,1107,69,1233,67,H,0,24,62,...,7,6,3,21,0.387097,0.440678,0.266667,0.181818,0.894737,0.785714
76638,2018,11,1112,101,1319,67,H,0,34,57,...,11,3,1,24,0.596491,0.396552,0.384615,0.307692,0.875000,0.566667


In [6]:
winning_df = reg_season_detailed_2018_pd[['Wteam', 'Wscore', 'Lscore', 'Wfgm', 'Wfga', 'Wfg%', 'Wfga3', 'Wfgm3', 'Wfg%3',
                                          'Wftm', 'Wfta', 'Wft%', 'Wor', 'Wdr', 'Wast', 'Wto', 'Wstl', 'Wblk', 'Wpf']]
losing_df = reg_season_detailed_2018_pd[['Lteam', 'Lscore', 'Wscore', 'Lfgm', 'Lfga', 'Lfg%', 'Lfga3', 'Lfgm3', 'Lfg%3',
                                         'Lftm', 'Lfta', 'Lft%', 'Lor', 'Ldr', 'Last', 'Lto', 'Lstl', 'Lblk', 'Lpf']]
winning_df.columns = ['team', 'for', 'against', 'fgm', 'fga', 'fg%', 'fga3', 'fgm3', 'fg%3', 'ftm', 'fta', 'ft%',
                      'or', 'dr', 'ast', 'to', 'stl', 'blk', 'pf']
losing_df.columns = ['team', 'for', 'against', 'fgm', 'fga', 'fg%', 'fga3', 'fgm3', 'fg%3', 'ftm', 'fta', 'ft%',
                     'or', 'dr', 'ast', 'to', 'stl', 'blk', 'pf']
season_stats_df = winning_df.append(losing_df)
season_stats = season_stats_df.groupby(['team'], as_index=False).mean()
print(season_stats)

     team        for    against        fgm        fga       fg%       fga3  \
0    1101  76.909091  69.818182  27.909091  57.363636  0.485853  21.090909   
1    1102  76.000000  68.625000  24.250000  56.500000  0.426541  25.875000   
2    1103  78.090909  70.181818  27.000000  56.909091  0.476906  25.363636   
3    1104  77.882353  65.235294  28.000000  55.000000  0.509194  19.176471   
4    1105  68.000000  59.500000  22.000000  47.500000  0.462955  19.500000   
5    1106  73.142857  62.142857  25.571429  57.142857  0.448883  21.000000   
6    1107  78.210526  66.789474  27.315789  55.789474  0.488995  14.789474   
7    1108  80.666667  63.500000  27.166667  53.666667  0.508379  20.333333   
8    1110  70.333333  62.666667  23.500000  50.000000  0.472556  23.000000   
9    1111  74.000000  64.900000  25.300000  56.800000  0.447471  24.100000   
10   1112  83.909091  69.136364  29.590909  55.863636  0.531295  18.318182   
11   1113  88.157895  72.947368  29.368421  59.947368  0.491287 

In [7]:
# perform clustering 
%matplotlib inline

num_clusters = 11
# stats to cluster on 
feature_columns = ['for', 'against', 'fga', 'fg%', 'fga3', 'fg%3', 'fta', 'ft%',
                      'or', 'dr', 'ast', 'to', 'stl', 'blk', 'pf']

cluster_columns = ['for', 'against', 'fg%', 'fg%3'] #best performing model 
to_cluster = season_stats[cluster_columns]

# normalize the stats 
x = to_cluster.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
scaled_df = pd.DataFrame(x_scaled)
scaled_df.columns = cluster_columns

# fit kmeans using to_cluster (non-normalized) or scaled_df(normalized)
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(to_cluster)

# extract data about clusters
cluster_map = pd.DataFrame()
cluster_map['Team_Id'] = season_stats.team.values
cluster_map['cluster'] = kmeans.labels_
pairs = []

#create matrix for cluster vs cluster percentages
for i in range(0,num_clusters):
    for j in range(0,num_clusters):
        if(i != j):
            pairs.append([i, j])

percentages = np.zeros((num_clusters,num_clusters))
for pair in pairs:
    cluster_1_wins = reg_season_detailed_2018_pd[
        reg_season_detailed_2018_pd['Wteam'].isin(cluster_map[cluster_map['cluster'] == pair[0]].Team_Id) & 
        reg_season_detailed_2018_pd['Lteam'].isin(cluster_map[cluster_map['cluster'] == pair[1]].Team_Id)
    ]
    cluster_2_wins = reg_season_detailed_2018_pd[
        reg_season_detailed_2018_pd['Wteam'].isin(cluster_map[cluster_map['cluster'] == pair[1]].Team_Id) & 
        reg_season_detailed_2018_pd['Lteam'].isin(cluster_map[cluster_map['cluster'] == pair[0]].Team_Id)
    ]
    percentages[pair[0], pair[1]] = cluster_1_wins.shape[0] / (cluster_1_wins.shape[0] + cluster_2_wins.shape[0])
    percentages[pair[1], pair[0]] = cluster_2_wins.shape[0] / (cluster_1_wins.shape[0] + cluster_2_wins.shape[0])

# get cluster centroids
centers = pd.DataFrame(kmeans.cluster_centers_)
centers.columns = cluster_columns
centers['cluster'] = ['cluster ' + str(i) for i in centers.index]
centers.set_index(['cluster'])
print(centers)
print(pd.DataFrame(percentages))

# get list of teams in each 
mm_teams = tourney_seeds_pd[tourney_seeds_pd['Season'] == 2018].Team_Id.values
mm_team_clusters = cluster_map[cluster_map['Team_Id'].isin(mm_teams)]
cluster_seeds = pd.merge(mm_team_clusters, tourney_seeds_pd[tourney_seeds_pd['Season'] == 2018])
team_cluster_seeds = pd.merge(cluster_seeds, teams_pd)
test = team_cluster_seeds.groupby('cluster')['Team_Name'].apply(lambda x: "%s" % ', '.join(x))
print(cluster_map.groupby(['cluster']).count())

          for    against       fg%      fg%3     cluster
0   84.638317  74.194613  0.482411  0.384917   cluster 0
1   79.751293  63.903266  0.494848  0.387261   cluster 1
2   69.213953  58.353382  0.454569  0.362467   cluster 2
3   75.035814  66.338765  0.466704  0.375101   cluster 3
4   83.169812  68.259300  0.494362  0.393800   cluster 4
5   78.595891  68.503658  0.478773  0.394640   cluster 5
6   72.861251  63.377827  0.470392  0.376556   cluster 6
7   91.552273  82.088805  0.501643  0.413459   cluster 7
8   79.841431  72.969111  0.473819  0.407282   cluster 8
9   87.845046  71.379604  0.501553  0.402122   cluster 9
10  76.421049  60.525192  0.480488  0.385121  cluster 10
          0         1         2         3         4         5         6   \
0   0.000000  0.428571  0.681818  0.513889  0.383333  0.519685  0.603448   
1   0.571429  0.000000  0.672414  0.857143  0.444444  0.708543  0.639098   
2   0.318182  0.327586  0.000000  0.385714  0.196429  0.478632  0.428571   
3   0.486111

In [10]:
#logreg model 

# construct feature matrix, and y vector 
feature_columns = ['for', 'against', 'fga', 'fg%', 'fga3', 'fg%3', 'fta', 'ft%',
                      'or', 'dr', 'ast', 'to', 'stl', 'blk', 'pf']
team_feature_columns = []
for col in feature_columns:
    team_feature_columns.append(col + '1')
for col in feature_columns:
    team_feature_columns.append(col + '2')
feature_matrix = season_stats[feature_columns]
cluster_feature_columns = []
for pair in pairs:
    cluster_feature_columns.append(",".join(str(x) for x in pair))

full_feature_columns = team_feature_columns + cluster_feature_columns
outcome = []
feature_matrix = []
diff_feature_matrix = []

for game in reg_season_detailed_2018_pd.itertuples():
    wteam_id = game.Wteam
    lteam_id = game.Lteam
    print(lteam_id)
    print(cluster_map[cluster_map['Team_Id'] == lteam_id])
    wteam_cluster = cluster_map[cluster_map['Team_Id'] == wteam_id].cluster.values[0]
    lteam_cluster = cluster_map[cluster_map['Team_Id'] == lteam_id].cluster.values[0]
    wteam_features = season_stats[season_stats['team'] == wteam_id][feature_columns].values
    lteam_features = season_stats[season_stats['team'] == lteam_id][feature_columns].values

    row1_features = np.append(wteam_features,lteam_features)
    row1_cluster = np.zeros((len(cluster_feature_columns), 1))
    cluster_str1 = str(wteam_cluster) + "," + str(lteam_cluster)
    if (wteam_cluster != lteam_cluster):
        row1_cluster[cluster_feature_columns.index(cluster_str1)] = 1
    row1_features = np.append(row1_features, row1_cluster)
    feature_matrix.append(row1_features)
    diff_feature_matrix.append(np.subtract(wteam_features,lteam_features)[0])
    outcome.append(1)
    
    row2_features = np.append(lteam_features,wteam_features)
    row2_cluster = np.zeros((len(cluster_feature_columns), 1))
    cluster_str2 = str(lteam_cluster) + "," + str(wteam_cluster)
    if (wteam_cluster != lteam_cluster):
        row2_cluster[cluster_feature_columns.index(cluster_str2)] = 1
    row2_features = np.append(row2_features, row2_cluster)
    feature_matrix.append(row2_features)
    diff_feature_matrix.append(np.subtract(lteam_features,wteam_features)[0])
    outcome.append(0)

1272
     Team_Id  cluster
163     1272        5
1233
     Team_Id  cluster
124     1233        0
1319
     Team_Id  cluster
208     1319        6
1226
     Team_Id  cluster
117     1226        3
1359
     Team_Id  cluster
247     1359        7
1313
     Team_Id  cluster
202     1313        5
1146
    Team_Id  cluster
40     1146        9
1288
     Team_Id  cluster
179     1288        1
1263
     Team_Id  cluster
154     1263        8
1180
    Team_Id  cluster
72     1180        8
1244
     Team_Id  cluster
135     1244        0
1366
     Team_Id  cluster
254     1366        7
1441
     Team_Id  cluster
328     1441        3
1373
     Team_Id  cluster
261     1373        3
1294
     Team_Id  cluster
184     1294        0
1170
    Team_Id  cluster
62     1170        3
1159
    Team_Id  cluster
51     1159        5
1463
     Team_Id  cluster
348     1463        4
1149
    Team_Id  cluster
42     1149        5
1123
    Team_Id  cluster
19     1123        4
1350
     Team_Id  cluster
238  

IndexError: index 0 is out of bounds for axis 0 with size 0